# **Procesamiento del habla**

Version 1.0

## Funcionalidades desbloqueadas por version:

*Version 1.0:*

- Entrada de consulta para un producto por vez
- Recomendacion para el cliente:
    - Algoritmo de ordenamiento basado en preferencias del usuario (dentro de la estructura gramatical)
    - Deteccion de precio
    - Caracteristicas particulares del producto

*Version 1.3*
- Entrada de consulta para mas de un producto
- Recomendaciones basadas en analisis de sentimiento

## Estructura de sub-etapas

---

```
├── Reconocimiento del lenguaje
│   ├── Captura de texto
│   └── Captura de audio
├── Procesamiento del lenguaje
│   ├── Preprocesamiento de texto
│   └── Analisis de texto
├── Gestion de respuesta
│   ├── Gestion del estado
│   ├── Motor de respuesta
│   └── Personalizacion
├── texto a voz
│   └── Conversion texto a voz

```


#### _Estructura operativa para cada sub-etapa:_

Se utilizaran las siguientes principales librerias por etapas, speechrecognition para reconocimiento de voz, pydub para transcripcion a texto, spacy para el procesamiento y analisis de texto y Google Text-to-Speech para la conversion de texto a voz.

Para la entrega de la recomendacion se tomaran los tokens lematizados (filtrados por adjetivos, sustantivos y proposiciones) de la entrada del usuario y de la columna de productos del dataframe, y se recorrera token por token entre cada una, para comprobar la similitud, entregando por ultimo, un diccionario con el indice del producto en la tabla original y el grado de considencia.

### Librerias y paquetes

#### *Entorno virtuales*

In [184]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

#### *Instalacion*

In [185]:
# Librerias de preprocesamiento
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub

# Librerias de reconocimiento de voz
!pip3 install pocketsphinx
!pip3 install SpeechRecognition
!pip3 install pyaudio

# Librerias de generacion de voz
!pip3 install gtts
# Librerias de analisis linguistico
!pip3 install spacy

In [186]:
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 1.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


#### *Importacion*

In [187]:
# Librerías para reconocimiento de voz y procesamiento de audio
import speech_recognition as sr
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment
import pyaudio

# Librerías para manipulación de archivos y compresión
import wave
import zipfile
import os
import subprocess
import io

# Librerías para procesamiento de datos y análisis
import numpy as np
import pandas as pd
import spacy
import re

# Librería para generación de voz
from gtts import gTTS

# Importación de funciones comunes a otros cuadernos
from funciones_comunes import common_functions

### Lectura de datos

In [188]:
# Defino el diccionario donde se almacenara la informacion de entrada y salida de la consulta
consulta_dict = {}

In [189]:
# Lectura y almacenado del dataframe
path_csv_rutas_verduras = '../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

## 1. Reconocimiento del Habla

___

### a. Captura de entradas de texto

In [190]:
# Conexion con api de fast-api
# aqui iria la conexion con un script encargado de conectar con la api de fast-api

In [191]:
# Funcion tentativa para recibir la entrada de texto desde fast-api

def entrada_texto(texto):
    texto = input('Ingresa brevemete el producto que deseas comprar y sus caracteristicas: ')
    return texto

### b. Captura de Audio

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [202]:
def entrada_voz():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        # Ajuste de sensibilidad y tiempo de pausa
        recognizer.energy_threshold = 300  # Ajustar según el entorno
        recognizer.pause_threshold = 1.0  # Ajustar según necesidad
        print("Por favor, habla ahora.")
        audio = recognizer.listen(source)
        
    try:
        texto = recognizer.recognize_google(audio, language='es-ES')
    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print("Error al conectarse con el servicio de Google: {0}".format(e))

    return texto

Por favor, habla ahora.


'bien Vamos a ver ahí si lo tomo yo tendría que estar hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando Hablando no se tendría que tener tendrá que ir Hablando Hablando Hablando Hablando Hablando Hablando Hablando vuelvo a hablar sigo hablando'

### c. Identificar tipo de entrada

In [203]:
# Recibo un tipo de entrada especifico y segun eso tomo un tipo de entrada o otro

entrada_tipo = 'audio'

if entrada_tipo == 'audio':
    texto = entrada_voz()
elif entrada_tipo == 'texto':
    texto = entrada_texto()

Por favor, habla ahora.


## 2. Procesamiento del Lenguaje Natural

___

In [204]:
# Cargar el modelo de spaCy para español

nlp = spacy.load("es_core_news_sm")

### Preprocesamiento de texto


In [196]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
'''

# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        #'toneladas', 'libras', 'onzas', 'quintales']

'''

"\n\n# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',\n                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',\n                        #'toneladas', 'libras', 'onzas', 'quintales']\n\n"

In [205]:
# Funcion encargada de procesar texto

def procesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[,\.;:!\-*#@$!+_%^&`~]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    thresgold = 1

    doc = nlp(texto)

    tokens = [token.texto for token in doc]
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    filtered_tokens = [
        token.texto for token in doc
        if token.texto not in stopwords
        and token.pos_ in ('NOUN', 'ADJ', 'ADP')
        and len(token.texto) > thresgold
        ]#and token.texto not in unidades_medida_peso
    lemmas = [token.lemma_ for token in doc if token.texto in filtered_tokens]
    pos_tags = [(token.texto, token.pos_) for token in doc if token.texto in filtered_tokens]

    return {
        "tokens": tokens,
        "lemmas": lemmas,
        "filtered_tokens": filtered_tokens,
        "pos_tags": pos_tags,
    }


**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

In [208]:
# texto analizado

texto_analizado = procesar_texto(texto)

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'texto'

**Tokenización**

In [ ]:
# tokens del texto analizado

tokens = texto_analizado['tokens']

**Eliminación de stop words.**


In [ ]:
# Tokens filtrados

tokens_filtrados = texto_analizado['filtered_tokens']

**Lematización y stemming**

In [ ]:
# lemas de tokens filtrados

lemmas_filtrados = texto_analizado['lemmas']

Etiquetado de estructuras gramaticales:

In [ ]:
# etiquetas gramaticales de los tokens filtrados

pos_tags_filtrados = texto_analizado['pos_tags']

### Análisis de texto


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [ ]:
# carga del conjunto de datos en dataframe

df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: procesar_texto(x)['lemmas'])

In [ ]:
# diccionario de considencias entre tokens lematizados de la columna y la entrada del usuario

dict_count_coincidences = {}

In [ ]:
# algoritmo para contar considencias entre tokens lematizados de la entrada del usuario y la columna/fila del dataframe

for index, row in df_frutas_verduras.iterrows():
    for token_producto in row['producto_tokens_lemmas']:
        if token_producto in lemmas_filtrados:
            if token_producto not in dict_count_coincidences:
                dict_count_coincidences[index] = 1
            else:
                dict_count_coincidences[index] += 1

In [ ]:
# Se transforma el diccionario a una seria de python para luego poder ordenarla por valor
series_ordered_count_coincidences = pd.Series(dict_count_coincidences)

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [ ]:
#para futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [ ]:
# para futuras versiones

## 3. Gestión del Diálogo

___

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


In [ ]:
consulta_dict['id_consulta'] = None
consulta_dict['id_cliente'] = None
consulta_dict['formato_consulta'] = None
consulta_dict['transcripción_audio'] = None
consulta_dict['entrada_texto'] = texto
consulta_dict['entrada_lemas_filtrados'] = lemmas_filtrados
consulta_dict['dict_producto_indice_considencias'] = dict_count_coincidences
consulta_dict['card_recomendacion'] = None
audio_recomendacion = None

### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


In [ ]:
# limpiando columna 'Precio' sacando signo de dolar y borrando los puntos del mil y remplazando la coma decimal por un punto

df_frutas_verduras['Precio'] = df_frutas_verduras['Precio'].apply(common_functions.limpiar_signo_peso)

In [ ]:
# Asegúrate de que series_ordered_count_coincidences es una Serie ordenada

series_ordered_count_coincidences = series_ordered_count_coincidences.sort_values(ascending=False)

# Inicialización de la recomendación
recomendacion = 'Los productos recomendados en base a su consulta y caracteristicas mensionadas son:\n'

# Lista para almacenar las recomendaciones
lista_string_reco_supers_prods = []

# Iterar sobre los índices de las coincidencias ordenadas
for count, x in enumerate(series_ordered_count_coincidences.index, start=1):
    # Extraer los detalles del producto usando el índice
    producto, supermercado, precio = df_frutas_verduras.loc[x, ['Producto', 'Supermercado', 'Precio']]
    
    # Añadir la recomendación a la lista
    lista_string_reco_supers_prods.append(f'En el lugar {count}: {producto} en {supermercado} a {precio} pesos\n')

# Concatenar todas las recomendaciones en un solo string
recomendacion += ''.join(lista_string_reco_supers_prods)

recomendacion = common_functions.redondear_numeros(recomendacion)

consulta_dict['card_recomendacion'] = recomendacion

Los productos recomendados en base a su consulta y caracteristicas mensionadas son:
En el lugar 1: Tomates secos 70 g. en Carrefour a 2229 pesos
En el lugar 2: Tomate seco 100 g. en Carrefour a 1799 pesos
En el lugar 3: Tomates secos 70 g. en Carrefour a 2229 pesos
En el lugar 4: Tomates Secos El Peoncito x 100 g. en La Anonima a 799 pesos
En el lugar 5: Tomates Peritas Deshidratados Nature Food x 150 g. en La Anonima a 4750 pesos
En el lugar 6: Tomates Deshidratados El Peoncito x 100 g. en La Anonima a 2150 pesos
En el lugar 7: Tomates Secos El Peoncito x 100 g. en La Anonima a 4000 pesos



Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


In [ ]:
# Para proximas versiones

### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

In [ ]:
# Para proximas versiones

## 4. texto a voz

___

### Conversión de texto a Voz (TTS): Utilizar servicios como Gogle texto-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


In [ ]:
tts = gTTS(texto=recomendacion, lang='es')

In [ ]:
# Guardar el audio en un objeto BytesIO
audio_buffer = io.BytesIO()
tts.write_to_fp(audio_buffer)

# Asegurarse de que el puntero esté al principio del buffer
audio_buffer.seek(0)

# Crear un diccionario y guardar el audio en él
consulta_dict = {'recomendacion_audio': audio_buffer}

In [ ]:
# Funciones separadas para las primeras etapas de reconocimiento del habla: audio, texto
def texto(texto):
    